# 1. Prepare SSH connection

In [7]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.DEBUG)
# Comment the follwing line to disable devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

In [8]:
# Generate plots inline
%pylab inline

import json
import os
import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace
#from trace_analysis import TraceAnalysis

# Support to configure and run RTApp based workloads
from wlgen import RTA, Ramp, Step, Pulse, Periodic

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


/home/lubaoquan/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['trace']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [9]:
# Let's use an example trace
#!pwd
#tracefile = os.path.join(res_dir, 'trace.dat')
platformfile = os.path.join("../../results/eas_acceptance/rtapp:energy_aware:early_and_migrators", 'platform.json')

In [10]:
# Trace events of interest
events_to_parse = [
                "sched_overutilized",
                "sched_energy_diff",
                "sched_load_avg_task",
                "sched_load_avg_cpu",
                "sched_migrate_task",
                "sched_switch"
]

# Platform description
with open(platformfile, 'r') as fh:
    platform = json.load(fh)

# Time range from the analysis
(t_min, t_max) = (0, None)

platform['nrg_model']['little']['cpu']['cap_max'] = 1024
platform['clusters']['little'] = [0, 1, 2, 3]
        
platform['nrg_model']['big']['cpu']['cap_max'] = 1024
platform['clusters']['big'] = [4, 5, 6, 7]
platform['cpus_count'] = 8

logging.info("CPUs max capacities:")
logging.info("   big: %5d (cpus: %s)",
             platform['nrg_model']['big']['cpu']['cap_max'],
             platform['clusters']['big'])
logging.info("LITTLE: %5d (cpus: %s)",
             platform['nrg_model']['little']['cpu']['cap_max'],
             platform['clusters']['little'])

04:11:45  INFO    : CPUs max capacities:
04:11:45  INFO    :    big:  1024 (cpus: [4, 5, 6, 7])
04:11:45  INFO    : LITTLE:  1024 (cpus: [0, 1, 2, 3])


In [11]:
# Load the LISA::Trace parsing module
from trace import Trace

results_dir="../../results/eas_acceptance"
dirlist=os.listdir(results_dir)

for filename in dirlist:
    #print filename
    filepath=os.path.join(results_dir, filename)
    res_dir=filepath+"/1"
    trace = Trace(platform, res_dir, events_to_parse, window=(t_min,t_max))
    trace.setXTimeRange(t_min, t_max)
    trace.analysis.frequency.plotClusterFrequencies()
    #trace.analysis.frequency.plotCPUFrequencyResidency(cpus=1)
    #trace.analysis.frequency.plotClusterFrequencyResidency(clusters='LITTLE')
    #trace.analysis.tasks.plotBigTasks(max_tasks=10, min_samples=100, min_utilization=10)
    #trace.analysis.tasks.plotTasks(tasks='rt-app-task0')
    #trace.analysis.tasks.plotBigTasks(max_tasks=10, min_samples=100, min_utilization=10)

04:11:45  DEBUG   : Loading [sched] events from trace in [../../results/eas_acceptance/rtapp:energy_aware:early_and_migrators/1]...
04:11:45  DEBUG   : Parsing events: ['sched_overutilized', 'sched_energy_diff', 'sched_load_avg_task', 'sched_load_avg_cpu', 'sched_migrate_task', 'sched_switch']
04:11:45  INFO    : Parsing FTrace format...
04:11:46  DEBUG   : Events found on trace:
04:11:46  DEBUG   :  - sched_load_avg_task
04:11:46  DEBUG   :  - sched_load_avg_cpu
04:11:46  DEBUG   :  - sched_migrate_task
04:11:46  DEBUG   :  - sched_overutilized
04:11:47  DEBUG   :  - sched_switch
04:11:47  DEBUG   :  - sched_energy_diff
04:11:47  DEBUG   : Lookup dataset for tasks...
04:11:47  DEBUG   : Lookup for task [swapper/3]...
04:11:47  DEBUG   :   task        swapper/3 found, pid: [0]
04:11:47  DEBUG   : Lookup for task [sh]...
04:11:47  DEBUG   :   task               sh found, pid: [25759, 25756, 25761, 25764, 25763, 25765, 25768, 25767, 25769, 25771, 25773, 25790, 25792, 25801, 25803, 25805,

Maximum estimated system energy: 5584


04:11:47  DEBUG   :   task      kworker/1:1 found, pid: [1188]
04:11:47  DEBUG   : Lookup for task [watchdog/0]...
04:11:47  DEBUG   :   task       watchdog/0 found, pid: [11]
04:11:47  DEBUG   : Lookup for task [watchdog/1]...
04:11:47  DEBUG   :   task       watchdog/1 found, pid: [12]
04:11:47  DEBUG   : Lookup for task [watchdog/2]...
04:11:47  DEBUG   :   task       watchdog/2 found, pid: [17]
04:11:47  DEBUG   : Lookup for task [watchdog/3]...
04:11:47  DEBUG   :   task       watchdog/3 found, pid: [22]
04:11:47  DEBUG   : Lookup for task [watchdog/4]...
04:11:47  DEBUG   :   task       watchdog/4 found, pid: [27]
04:11:47  DEBUG   : Lookup for task [watchdog/5]...
04:11:47  DEBUG   :   task       watchdog/5 found, pid: [32]
04:11:47  DEBUG   : Lookup for task [watchdog/6]...
04:11:47  DEBUG   :   task       watchdog/6 found, pid: [37]
04:11:47  DEBUG   : Lookup for task [watchdog/7]...
04:11:47  DEBUG   :   task       watchdog/7 found, pid: [42]
04:11:47  DEBUG   : Lookup for ta

Maximum estimated system energy: 5584


04:11:49  DEBUG   :   task     kworker/0:1H found, pid: [1732]
04:11:49  DEBUG   : Lookup for task [kworker/u16:3]...
04:11:49  DEBUG   :   task    kworker/u16:3 found, pid: [25402]
04:11:49  DEBUG   : Lookup for task [AlarmManager]...
04:11:49  DEBUG   :   task     AlarmManager found, pid: [2192]
04:11:49  DEBUG   : Lookup for task [ActivityManager]...
04:11:49  DEBUG   :   task  ActivityManager found, pid: [2170]
04:11:49  DEBUG   : Lookup for task [PowerManagerSer]...
04:11:49  DEBUG   :   task  PowerManagerSer found, pid: [2179]
04:11:49  DEBUG   : Lookup for task [system_server]...
04:11:49  DEBUG   :   task    system_server found, pid: [2150]
04:11:49  DEBUG   : Lookup for task [Binder:2245_2]...
04:11:49  DEBUG   :   task    Binder:2245_2 found, pid: [2282]
04:11:49  DEBUG   : Lookup for task [ndroid.systemui]...
04:11:49  DEBUG   :   task  ndroid.systemui found, pid: [2245]
04:11:49  DEBUG   : Lookup for task [Binder:2150_6]...
04:11:49  DEBUG   :   task    Binder:2150_6 found,

Maximum estimated system energy: 5584


04:11:51  DEBUG   :   task       watchdog/0 found, pid: [11]
04:11:51  DEBUG   : Lookup for task [watchdog/1]...
04:11:51  DEBUG   :   task       watchdog/1 found, pid: [12]
04:11:51  DEBUG   : Lookup for task [watchdog/2]...
04:11:51  DEBUG   :   task       watchdog/2 found, pid: [17]
04:11:51  DEBUG   : Lookup for task [watchdog/3]...
04:11:51  DEBUG   :   task       watchdog/3 found, pid: [22]
04:11:51  DEBUG   : Lookup for task [watchdog/4]...
04:11:51  DEBUG   :   task       watchdog/4 found, pid: [27]
04:11:51  DEBUG   : Lookup for task [watchdog/5]...
04:11:51  DEBUG   :   task       watchdog/5 found, pid: [32]
04:11:51  DEBUG   : Lookup for task [watchdog/6]...
04:11:51  DEBUG   :   task       watchdog/6 found, pid: [37]
04:11:51  DEBUG   : Lookup for task [watchdog/7]...
04:11:51  DEBUG   :   task       watchdog/7 found, pid: [42]
04:11:51  DEBUG   : Lookup for task [mmcqd/0]...
04:11:51  DEBUG   :   task          mmcqd/0 found, pid: [1196]
04:11:51  DEBUG   : Lookup for task 

Maximum estimated system energy: 5584


04:11:54  DEBUG   : Lookup for task [swapper/7]...
04:11:54  DEBUG   :   task        swapper/7 found, pid: [0]
04:11:54  DEBUG   : Lookup for task [shell srvc 2596]...
04:11:54  DEBUG   :   task  shell srvc 2596 found, pid: [25962]
04:11:54  DEBUG   : Lookup for task [rt-app]...
04:11:54  DEBUG   :   task           rt-app found, pid: [25982, 25983, 25986, 25985, 25984, 25987, 25988, 25990, 25989]
04:11:54  DEBUG   : Lookup for task [task_taskpack0]...
04:11:54  DEBUG   :   task   task_taskpack0 found, pid: [25983]
04:11:54  DEBUG   : Lookup for task [task_taskpack3]...
04:11:54  DEBUG   :   task   task_taskpack3 found, pid: [25986]
04:11:54  DEBUG   : Lookup for task [task_taskpack2]...
04:11:54  DEBUG   :   task   task_taskpack2 found, pid: [25985]
04:11:54  DEBUG   : Lookup for task [task_taskpack1]...
04:11:54  DEBUG   :   task   task_taskpack1 found, pid: [25984]
04:11:54  DEBUG   : Lookup for task [task_taskpack4]...
04:11:54  DEBUG   :   task   task_taskpack4 found, pid: [25987]
